In [180]:
# -*- coding: utf-8 -*-
'''
По ИНН качает данные с зайта https://zakupki.gov.ru/
Данные качает через TorBrowser https://www.torproject.org/download/ (качать только тут), тк тор подменяет IP.s
'''

import socket,requests,socks,os
# from random import randint
import pandas as pd
from fake_useragent import UserAgent
# from glob import glob
# from multiprocessing import Process,cpu_count
# from threading import Thread
from bs4 import BeautifulSoup as bs
import re,time
from datetime import datetime
from urllib.parse import unquote


def GetText(x):
    x = re.sub('\n+','',x)
    x = re.sub('\r+','',x)
    x = re.sub('\t+','',x)
    x = re.sub('\s+',' ',x)
    return x

def FindVal(x):
    for xx in x:
        if  not xx.isdigit():
           if  xx != '.' and xx != ',':
               return xx

try:os.mkdir('allFiles')
except:pass
zakupki_tab = pd.DataFrame({})
documents_tab = pd.DataFrame({})
protocols_tab = pd.DataFrame({})

#socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
#socket.socket = socks.socksocket

inns = ['7702770003']#, '7736641983', '7801392271']
url = """https://zakupki.gov.ru/epz/order/extendedsearch/results.html?searchString={0}&morphology=on&search-filter=%D0%94%D0%B0%D1%82%D0%B5+%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D1%8F&pageNumber={1}&sortDirection=false&recordsPerPage=_500&showLotsInfoHidden=false&sortBy=UPDATE_DATE&fz223=on&pc=on&currencyIdGeneral=-1&publishDateFrom=01.01.2019&publishDateTo=31.03.2020&selectedSubjectsIdNameHidden=%7B%7D"""
dataurl = {'searchString': '{0}','morphology': 'on'
,'search-filter': 'Дате размещения','pageNumber': '{0}'
,'sortDirection': 'false','recordsPerPage': '_500'
,'showLotsInfoHidden': 'false','sortBy': 'UPDATE_DATE'
,'fz223': 'on','pc': 'on'
,'currencyIdGeneral': '-1','publishDateFrom': '01.01.2019'
,'publishDateTo': '31.03.2020' ,'selectedSubjectsIdNameHidden': '{}'}


linn = len(inns)
counterer = 0
for inn in inns:
    if counterer < 1:
        counterer+=1
        page = 1
        innurl = url.format(inn,page)
        dataurl['searchString'] = dataurl['searchString'].format(inn)
        dataurl['pageNumber'] = dataurl['pageNumber'].format(page)
        response = requests.get(innurl, data=dataurl, headers={'User-Agent': UserAgent().chrome})
        html = response.text
        html = bs(html,'lxml')
        count = html.find("div", {"class": "search-results__total"})
        count = int(re.findall('[0-9]+',count.text)[0])
        print(linn,inn,count)
        linn-=1
        if count==0:
            allInfo = {'ИНН':inn,
                       'Страница':page,
                       'Кол_закупок_на_стр':0,
                       'Пор_номер_закупки':0,
                        'Номер_закупки':'',
                       'Тип_закупки':'',
                       'Объект_закупки':'',
                       'Заказчик':'',
                       'Начальная цена':'',
                       'Размещено':'',
                       'Обновлено':'',
                       'Окончание_подачи_заявок':'',
                       'Документы_URL':'',
                       'Протоколы_URL':''}
            zakupki_tab = zakupki_tab.append(allInfo,ignore_index=True)
            time.sleep(1)
            continue
        html_zakupka_block = html.findAll("div", {"class": 'row no-gutters registry-entry__form mr-0'})
        num_zak=0
        lbloks = len(html_zakupka_block)
        for block in html_zakupka_block:
            print('\r %s' % lbloks,end='')
            lbloks-=1
            num_zak+=1
            numb = block.find("div", {"class": 'registry-entry__header-mid__number'})
            if numb is None:
                numb=''
            else:
                numb = GetText(numb.text)
            typez = block.find("div", {"class": 'registry-entry__header-top__title text-truncate'})
            if typez is None:
                typez=''
            else:
                typez = GetText(typez.text)
            objz = block.find("div", {"class": 'registry-entry__body-value'})
            if objz is None:
                objz=''
            else:
                objz = GetText(objz.text)

            zakazchik = block.find("div", {"class": 'registry-entry__body-href'})
            if zakazchik is None:
                zakazchik=''
            else:
                zakazchik = GetText(zakazchik.text)
            start_price = block.find("div", {"class": 'price-block__value'})
            if start_price is None:
                start_price=''
                valut=''
            else:
                start_price = GetText(start_price.text)
                start_price = start_price.replace(u'\xa0',u'')
                start_price = start_price.replace(' ','').replace(',','.')
                valut = FindVal(start_price)
                start_price = float(start_price.split(valut)[0])
            dates = block.findAll("div", {"class": 'data-block__value'})
            dates = [GetText(x.text) for x in dates]#размещено/обновлено/окончание подачи заявок
            rasmesheno = datetime.strptime(dates[0],'%d.%m.%Y')
            update = datetime.strptime(dates[1],'%d.%m.%Y')
            okon_podach_zayav = datetime.strptime(dates[2],'%d.%m.%Y')
            srcs = block.findAll("div", {"class": 'href d-flex'})
            srcs =[srcs[0],srcs[-1]]
            docs = srcs[0].a.attrs['href']
            try:
                protocol = srcs[1].a.attrs['href']
            except:protocol=''
            allInfo = {'ИНН':inn,
                       'Страница':page,
                       'Кол_закупок_на_стр':count,
                       'Пор_номер_закупки':num_zak,
                        'Номер_закупки':numb,
                       'Тип_закупки':typez,
                       'Объект_закупки':objz,
                       'Заказчик':zakazchik,
                       'Начальная цена':start_price,
                       'Валюта':valut,
                       'Размещено':rasmesheno,
                       'Обновлено':update,
                       'Окончание_подачи_заявок':okon_podach_zayav,
                       'Документы_URL':docs,
                       'Протоколы_URL':protocol}
            zakupki_tab = zakupki_tab.append(allInfo,ignore_index=True)

            docs_req  = requests.get(docs, headers={'User-Agent': UserAgent().chrome})
            docs_html = docs_req.text
            docs_html = bs(docs_html,'lxml')
            docs_h2 = docs_html.findAll('h2')
            docs_h2 =[x.text for x in docs_h2]
            docs_tab = docs_html.findAll("div", {"class": 'noticeTabBoxWrapper'})

            for h2 in range(len(docs_h2)):
                dh2 = docs_h2[h2]
                dt = docs_tab[h2]
                dt = dt.findAll("a", {"class": 'epz_aware'})
                if len(dt)==0:continue
                for file in dt:
                    href = 'https://zakupki.gov.ru'+file.attrs['href']
                    namef = file.attrs['title']
                    text = GetText(file.text)
                    downlodad = requests.get(href, allow_redirects=True)
                    filename = downlodad.headers['Content-Disposition']
                    try:
                        filename = filename.split('filename="')[-1].split('";')[0]
                        filename = unquote(filename)#декодируем название файла

                        filename = filename.replace(':','_').replace('"','').replace("'",'')


                        filename_pois = '%s_%s_%s' % (inn,numb,filename)

                        filePath = os.path.join('allFiles', filename_pois)
                        hiperlink = '!=ГИПЕРССЫЛКА("%s";"%s")' % (filePath,filename)
    #                     open(filePath, 'wb').write(downlodad.content)
                    except:
                        print(inn,filename,'ERROR')
                        continue

                    docs_dict = {'ИНН':inn,
                                 'Номер_закупки':numb,
                                 'Заказчик':zakazchik,
                                 'Ссылка_на_док':href,
                                 'Тип_документа':dh2,
                                 'Имя_из_HTML':text,
                                 'Имя_файла':filename,
                                 'Ссыдка_на_файл':hiperlink}
                    documents_tab = documents_tab.append(docs_dict,ignore_index=True)
            if protocol=='':continue
            protocol_req =  requests.get(protocol, headers={'User-Agent': UserAgent().chrome})
            protocol_html = protocol_req.text
            protocol_html = bs(protocol_html,'lxml')

            desc = protocol_html.findAll("td", {"class": 'descriptTenderTd'})
            if len(desc)!=0:
                for dc in desc:
                    name_prot = dc.find('span',{ "class":"protocolName"})
                    name_prot_end = GetText(name_prot.text)
                    pachat_forms = dc.find('li')
                    href = pachat_forms.attrs['onclick']
                    href = re.findall('[(].+?[)]',href)[0]
                    href = 'https://zakupki.gov.ru'+href.replace('(','').replace(')','').replace("'",'')
                    href = href+'#tabs-1'
                    name_prot_end = name_prot_end.replace(':','_').replace('"','').replace("'",'')

                    downlodad = requests.get(href)
                    filename_pois = '%s_%s_%s.html' % (inn,numb,name_prot_end)
                    filePath = os.path.join('allFiles', filename_pois)
                    hiperlink = '!=ГИПЕРССЫЛКА("%s";"%s")' % (filePath,filename_pois)
#                     open(filePath, 'wb').write(downlodad.content)

                    protocolentityid = name_prot.attrs['protocolentityid']
                    protocols_docs = 'https://zakupki.gov.ru/223/purchase/public/purchase/protocol/ip/documents.html?activeTab=6&protocolInfoId={0}&mode=view'.format(protocolentityid)

                    protocol_dict = {
                        'ИНН':inn,
                        'Номер_закупки':numb,
                        'Заказчик':zakazchik,
                        'Наименование_документа':name_prot_end,
                        'Ссылка_на_док':href,
                        'Ссыдка_на_файл':hiperlink
                        }

                    protocols_tab = protocols_tab.append(protocol_dict,ignore_index=True)


                    protocol_docs_req =  requests.get(protocols_docs, headers={'User-Agent': UserAgent().chrome})
                    protocol_docs_html = protocol_docs_req.text
                    protocol_docs_html = bs(protocol_docs_html,'lxml')


                    pr_tab = protocol_docs_html.find('tbody')
                    if pr_tab is None:
                        continue
                    if pr_tab.td is None:
                        continue
                    pr_tab = [x for x in pr_tab.td.children if x!='\n'][1:]#ссылки на
                    for prt in pr_tab:
                        href2 = 'https://zakupki.gov.ru'+prt.attrs['href']

                        try:
                            downlodad2 = requests.get(href2)
                        except:
                            time.slep(1)
                            downlodad2 = requests.get(href2)

                        filename2 = downlodad2.headers['Content-Disposition']
                        filename2 = filename2.split('filename="')[-1].split('";')[0]
                        filename2 = unquote(filename2)#протокол из документов(на основании которого делался протокол из сведений о протоколах)
                        print(filename2)
                        filename2 = filename2.replace(':','_').replace('"','').replace("'",'')

                        filename_pois2 = '%s_%s_%s.html' % (inn,numb,filename2)
                        filePath2 = os.path.join('allFiles', filename_pois2)
                        hiperlink2 = '!=ГИПЕРССЫЛКА("%s";"%s")' % (filePath,filename2)


                        protocol_docs_dict = {
                            'ИНН':inn,
                            'Номер_закупки':numb,
                            'Заказчик':zakazchik,
                            'Наименование_документа':filename2,
                            'Ссылка_на_док':href2,
                            'Ссыдка_на_файл':hiperlink2}
                        protocols_tab = protocols_tab.append(protocol_docs_dict,ignore_index=True)
        print()



    zakupki_tab = zakupki_tab[['ИНН','Страница','Кол_закупок_на_стр',
                       'Пор_номер_закупки','Номер_закупки',
                       'Тип_закупки','Объект_закупки',
                       'Заказчик','Начальная цена',
                       'Размещено','Обновлено',
                       'Окончание_подачи_заявок',
                       'Документы_URL','Протоколы_URL']]
    documents_tab = documents_tab[['ИНН','Номер_закупки','Заказчик',
                                 'Ссылка_на_док','Тип_документа',
                                 'Имя_из_HTML','Имя_файла','Ссыдка_на_файл']]

    protocols_tab = protocols_tab[[ 'ИНН','Номер_закупки',
                            'Заказчик',
                            'Наименование_документа',
                            'Ссылка_на_док',
                            'Ссыдка_на_файл']]
    
writer = pd.ExcelWriter('PR_weatherGDDid.xlsx', engine='xlsxwriter') 

# Write each dataframe to a different worksheet.
zakupki_tab.to_excel(writer,'Закупки3.xlsx', index=False)     
documents_tab.to_excel(writer, 'Документы3.xlsx', index=False)    

# Close the Pandas Excel writer and output the Excel file.
writer.save()
#     zakupki_tab.to_excel('Закупки3.xlsx',index=False)
#     documents_tab.to_excel('Документы3.xlsx',index=False)
#     protocols_tab.to_excel('Протоколы3.xlsx',index=False)

1 7702770003 39
 38протокол итоговый.docx
Протокол гсм.pdf
 37

KeyboardInterrupt: 